In [1]:
import pandas as pd

In [2]:
file_path = "D:/Verifikasi Kategori Portfolio Antasena Kredit/10-2024/RISK 11-2024/LOANDB OKT 2024 V1.0 (BEFORE CKPN).xlsx"
cols = ["ACCOUNT_NUMBER","CIF_MASTER","NAME","DEBTOR_CATEGORY","PORTOFOLIO_CATEGORY",
        "RESTRUCTURED_OR_NORMAL","TYPE_OF_LOAN","START","END","COLLECTIBILITY","CURRENT_LOAN_OUTSTANDING",
        "DEBTOR_CLASSIFICATION","PRODUCT_NEW"]

loandb = pd.read_excel(file_path, sheet_name="LoanDB", usecols=cols)

In [3]:
file_path = "D:/Verifikasi Kategori Portfolio Antasena Kredit/10-2024/RISK 11-2024/Report_DataPelaporan_Agunan_KreditPembiayaan.xlsx"
cols = ["NOMOR REKENING", "NOMOR AGUNAN"]

agunankreditpembiayaan = pd.read_excel(file_path, sheet_name="Report_DataPelaporan_Agunan_Kre", usecols=cols, skiprows=6)

In [79]:
agunankreditpembiayaan.columns

Index(['NOMOR REKENING', 'NOMOR AGUNAN'], dtype='object')

In [4]:
file_path = "D:/Verifikasi Kategori Portfolio Antasena Kredit/10-2024/RISK 11-2024/Report_DataPelaporan_Agunan.xlsx"
cols = ["NOMOR AGUNAN", "JENIS AGUNAN", "NILAI AGUNAN"]

agunan = pd.read_excel(file_path, sheet_name="Report_DataPelaporan_Agunan", usecols=cols, skiprows=6)

In [78]:
agunan.columns

Index(['NOMOR AGUNAN', 'JENIS AGUNAN', 'NILAI AGUNAN'], dtype='object')

In [5]:
file_path = "D:/Verifikasi Kategori Portfolio Antasena Kredit/10-2024/RISK 11-2024/Esinsial 1.xlsx"

esinsial_1 = pd.read_excel(file_path)

In [6]:
file_path = "D:/Verifikasi Kategori Portfolio Antasena Kredit/10-2024/RISK 11-2024/Esinsial 2.xlsx"

esinsial_2 = pd.read_excel(file_path)

In [7]:
agunan_gabungan = pd.merge(
    agunankreditpembiayaan, 
    agunan, 
    on='NOMOR AGUNAN',  # Kolom untuk join
    how='inner'  # Tipe join (inner join)
)

In [8]:
# Daftar nilai untuk filter
filter_jenis_agunan = [
    'AN020101', 'AN02010201', 'AN02010202', 'AN02010203', 
    'AN02010204', 'AN02010299', 'AN02010301', 'AN02010302'
]

# Melakukan filter pada kolom JENIS AGUNAN dan membuat salinan DataFrame
agunan_property_only = agunan_gabungan[agunan_gabungan['JENIS AGUNAN'].isin(filter_jenis_agunan)].copy()


In [9]:
# Fungsi untuk menentukan nilai 'CEK AGUNAN'
def cek_agunan(jenis_agunan):
    if jenis_agunan in ['AN020101', 'AN02010201', 'AN02010202', 'AN02010203', 'AN02010204', 'AN02010299']:
        return 'Beragun Properti Komersial'
    elif jenis_agunan in ['AN02010301', 'AN02010302']:
        return 'Beragun Rumah Tinggal'
    else:
        return 'Take Out'

# Tambahkan kolom 'CEK AGUNAN' menggunakan apply
agunan_property_only['CEK AGUNAN'] = agunan_property_only['JENIS AGUNAN'].apply(cek_agunan)
agunan_property_only = agunan_property_only.reset_index(drop=True)

In [10]:
# Agregasi nilai 'NILAI AGUNAN' berdasarkan 'NOMOR REKENING' dan 'CEK AGUNAN'
agg = agunan_property_only.groupby(['NOMOR REKENING', 'CEK AGUNAN'])['NILAI AGUNAN'].sum().unstack(fill_value=0)

# Gabungkan hasil agregasi ke DataFrame asli, reset index untuk menggabungkan dengan kolom NOMOR REKENING
agunan_property_only = agunan_property_only.merge(agg[['Beragun Properti Komersial', 'Beragun Rumah Tinggal']], 
                                                  on='NOMOR REKENING', 
                                                  how='left')

In [11]:
# Fungsi untuk menentukan nilai FINAL COLLATERAL
def final_collateral(row):
    if row['Beragun Properti Komersial'] > row['Beragun Rumah Tinggal']:
        return 'Beragun Properti Komersial'
    else:
        return 'Beragun Rumah Tinggal'

# Menambahkan kolom FINAL COLLATERAL menggunakan apply
agunan_property_only['FINAL COLLATERAL'] = agunan_property_only.apply(final_collateral, axis=1)

In [13]:
agunan_property_only_cleaned = agunan_property_only.drop_duplicates(subset='NOMOR REKENING', keep='first')

In [60]:
# Melakukan left join pada loandb dan agunan
worksheet_katpor = pd.merge(
    loandb,
    agunan_property_only_cleaned,
    left_on='ACCOUNT_NUMBER',
    right_on='NOMOR REKENING',
    how='left'
)

In [61]:
# Menambahkan kolom 'CEK_AGUNAN_NEW'
worksheet_katpor['CEK_AGUNAN_NEW'] = worksheet_katpor['FINAL COLLATERAL'].apply(
    lambda x: x if pd.notna(x) else 'Non Beragun Properti'
)

In [62]:
# Pastikan tidak ada NaN sebelum konversi
worksheet_katpor['CIF_MASTER'] = pd.to_numeric(worksheet_katpor['CIF_MASTER'], errors='coerce').fillna(0).astype(int)
#esinsial_1['ID_DEBITUR'] = pd.to_numeric(esinsial_1['ID_DEBITUR'], errors='coerce').fillna(0).astype(int)

# Melakukan left join antara 'worksheet_katpor' dan 'esinsial_1'
worksheet_katpor = pd.merge(
    worksheet_katpor,
    esinsial_1,
    left_on='CIF_MASTER',
    right_on='ID_DEBITUR',
    how='left'
)

# Menambahkan kolom 'CEK_KATPOR_ESSENTIAL'
worksheet_katpor['CEK_KATPOR_ESSENTIAL'] = worksheet_katpor['KATPOR UPDATE'].apply(
    lambda x: x if pd.notna(x) else 'Cek'
)

In [63]:
# Daftar kolom yang akan dihapus
columns_to_drop = [
    'NOMOR REKENING', 'NOMOR AGUNAN', 'JENIS AGUNAN', 'NILAI AGUNAN', 'CEK AGUNAN',
    'Beragun Properti Komersial', 'Beragun Rumah Tinggal', 'FINAL COLLATERAL', 
    'NOMOR_REKENING', 'ID_DEBITUR', 'NAMA', 'KATPOR UPDATE', 'Sandi Jenis Debitur'
]

# Menghapus kolom dari DataFrame
worksheet_katpor = worksheet_katpor.drop(columns=columns_to_drop, errors='ignore')

In [64]:
# Langkah 1: Lakukan join sementara dengan esinsial_2
joined_df = pd.merge(
    worksheet_katpor, 
    esinsial_2[['Sandi Referensi', 'Kode']], 
    left_on='DEBTOR_CLASSIFICATION', 
    right_on='Sandi Referensi', 
    how='left'
)

# Langkah 2: Tambahkan kolom 'CEK_GOLDEB' berdasarkan logika
joined_df['CEK_GOLDEB'] = joined_df.apply(
    lambda row: row['CEK_KATPOR_ESSENTIAL'] 
    if row['CEK_KATPOR_ESSENTIAL'] != 'Cek' 
    else (row['Kode'] if pd.notna(row['Kode']) else 'Cek'), 
    axis=1
)

# Hapus kolom tambahan jika tidak dibutuhkan lagi
joined_df.drop(columns=['Sandi Referensi', 'Kode'], inplace=True)

# Hasil akhir
worksheet_katpor = joined_df

In [65]:
worksheet_katpor['START'] = pd.to_datetime(
    worksheet_katpor['START'], 
    errors='coerce'
)

# Tangani nilai tanggal di luar rentang dengan NaT
worksheet_katpor['END'] = pd.to_datetime(
    worksheet_katpor['END'], 
    errors='coerce'
)

# Cari tanggal terbesar di kolom 'START' dan 'END'
max_start_date = worksheet_katpor['START'].max()
max_end_date = worksheet_katpor['END'].max()

# Isi nilai NaT dengan tanggal terbesar
worksheet_katpor['START'] = worksheet_katpor['START'].fillna(max_start_date)
worksheet_katpor['END'] = worksheet_katpor['END'].fillna(max_end_date)

In [66]:
# Fungsi untuk menghitung nilai kolom 'JANGKA'
def calculate_jangka(row):
    if row['CEK_GOLDEB'] == "Cek Jangka - 17/18" and (row['END'] - row['START']).days > 90:
        return 18
    elif row['CEK_GOLDEB'] == "Cek Jangka - 17/18" and (row['END'] - row['START']).days <= 90:
        return 17
    elif row['CEK_GOLDEB'] == "Cek Jangka - 14/15" and (row['END'] - row['START']).days > 90:
        return 15
    elif row['CEK_GOLDEB'] == "Cek Jangka - 14/15" and (row['END'] - row['START']).days <= 90:
        return 14
    else:
        return row['CEK_GOLDEB']

# Menambahkan kolom 'JANGKA'
worksheet_katpor['JANGKA'] = worksheet_katpor.apply(calculate_jangka, axis=1)

In [67]:
# Fungsi untuk menghitung nilai kolom 'PENSIUN'
def calculate_pensiun(row):
    if row['JANGKA'] == "Cek" and row['PRODUCT_NEW'] == "PERSONAL":
        return 40
    else:
        return row['JANGKA']

# Menambahkan kolom 'PENSIUN'
worksheet_katpor['PENSIUN'] = worksheet_katpor.apply(calculate_pensiun, axis=1)

In [68]:
# Melakukan left join untuk mendapatkan kolom 'KATPOR UPDATE' dari esinsial_1
worksheet_katpor = pd.merge(
    worksheet_katpor,
    esinsial_1[['ID_DEBITUR', 'KATPOR UPDATE']],
    left_on='CIF_MASTER',
    right_on='ID_DEBITUR',
    how='left'
)

# Menambahkan kolom 'CEK TANAH&ESSENTIAL' dengan logika formula
def cek_tanah_essential(row):
    if row['PENSIUN'] == "Cek" and row['RESTRUCTURED_OR_NORMAL'] == 63:
        return row['RESTRUCTURED_OR_NORMAL']
    elif row['PENSIUN'] == "Cek":
        return row['KATPOR UPDATE'] if pd.notna(row['KATPOR UPDATE']) else row['PENSIUN']
    else:
        return row['PENSIUN']

worksheet_katpor['CEK TANAH&ESSENTIAL'] = worksheet_katpor.apply(cek_tanah_essential, axis=1)

# Hapus kolom 'ID_DEBITUR' dan 'KATPOR UPDATE' jika tidak diperlukan lagi
worksheet_katpor.drop(['ID_DEBITUR', 'KATPOR UPDATE'], axis=1, inplace=True)

In [69]:
# Menambahkan kolom baru dengan logika formula
def cek_collateral_logic(row):
    if row['CEK TANAH&ESSENTIAL'] == "Cek" and row['CEK_AGUNAN_NEW'] == "Beragun Rumah Tinggal":
        return 41
    elif row['CEK TANAH&ESSENTIAL'] == "Cek" and row['CEK_AGUNAN_NEW'] == "Beragun Properti Komersial":
        return 42
    else:
        return row['CEK TANAH&ESSENTIAL']

# Menambahkan kolom CEK_COLLATERAL
worksheet_katpor['CEK_COLLATERAL'] = worksheet_katpor.apply(cek_collateral_logic, axis=1)


In [70]:
# Menambahkan kolom baru dengan logika formula
def cek_sme_logic(row):
    if (
        row['CEK_COLLATERAL'] == "Cek" and 
        (row['DEBTOR_CATEGORY'] in ["UM", "UK"] or 
         row['DEBTOR_CLASSIFICATION'] in ["S14", "S24BL"])
    ):
        return 36
    else:
        return row['CEK_COLLATERAL']

# Menambahkan kolom CEK_SME
worksheet_katpor['CEK_SME'] = worksheet_katpor.apply(cek_sme_logic, axis=1)


In [71]:
# Menambahkan kolom baru dengan logika formula
def cek_default_logic(row):
    if (
        (row['COLLECTIBILITY'] > 2 or row['RESTRUCTURED_OR_NORMAL'] == 1) and
        row['CEK_SME'] == 41 and
        row['TYPE_OF_LOAN'] == 3
    ):
        return 53
    elif (
        (row['COLLECTIBILITY'] > 2 or row['RESTRUCTURED_OR_NORMAL'] == 1) and
        (
            row['CEK_SME'] != 41 or
            (row['CEK_SME'] == 41 and row['TYPE_OF_LOAN'] != 3)
        )
    ):
        return 54
    else:
        return row['CEK_SME']

# Menambahkan kolom CEK_DEFAULT
worksheet_katpor['CEK_DEFAULT'] = worksheet_katpor.apply(cek_default_logic, axis=1)


In [72]:
# Menambahkan kolom FINAL berdasarkan kondisi
worksheet_katpor['FINAL'] = worksheet_katpor['CEK_DEFAULT'].apply(
    lambda x: 35 if x == "Cek" else x
)

worksheet_katpor['FINAL'] = worksheet_katpor['FINAL'].astype('int64')

In [73]:
# Menambahkan kolom 'NOTE' dengan formula yang diinginkan
worksheet_katpor['NOTE'] = worksheet_katpor.apply(
    lambda row: 'True' if row['FINAL'] == row['PORTOFOLIO_CATEGORY'] else 'False',
    axis=1
)


In [76]:
#worksheet_katpor.to_excel('worksheet_katpor.xlsx', index=False)

In [77]:
worksheet_katpor.columns

Index(['ACCOUNT_NUMBER', 'CIF_MASTER', 'NAME', 'DEBTOR_CLASSIFICATION',
       'DEBTOR_CATEGORY', 'RESTRUCTURED_OR_NORMAL', 'TYPE_OF_LOAN',
       'COLLECTIBILITY', 'PORTOFOLIO_CATEGORY', 'CURRENT_LOAN_OUTSTANDING',
       'START', 'END', 'PRODUCT_NEW', 'CEK_AGUNAN_NEW', 'CEK_KATPOR_ESSENTIAL',
       'CEK_GOLDEB', 'JANGKA', 'PENSIUN', 'CEK TANAH&ESSENTIAL',
       'CEK_COLLATERAL', 'CEK_SME', 'CEK_DEFAULT', 'FINAL', 'NOTE'],
      dtype='object')